<a href="https://colab.research.google.com/github/rootUJ99/hf_transformer_diffusers/blob/main/transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install safetensors torch huggingface_hub

In [4]:
from transformers import pipeline
import transformers
print(transformers.__version__)

4.31.0


pipeline -> dont know what is does

In [4]:
generator = pipeline(task="automatic-speech-recognition", model="openai/whisper-small", batch_size=2)

In [5]:
# generator("./sample_data/jfk.wav")

In [6]:
generator(
    [
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/mlk.flac",
        "https://huggingface.co/datasets/Narsil/asr_dummy/resolve/main/1.flac",
    ]
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'text': ' I have a dream that one day this nation will rise up and live out the true meaning of its creed.'},
 {'text': ' He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat mutton pieces to be ladled out in thick, peppered, flour-fattened sauce.'}]

### Trying safe tensor (can skip this block for if just looking for )

In [7]:
import safetensors
import os
import datetime
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
import torch

sf_model = hf_hub_download("gpt2", filename="model.safetensors")
pt_model = hf_hub_download("gpt2", filename="pytorch_model.bin")

start_sf = datetime.datetime.now()
load_file(sf_model)
loadtime_sf = datetime.datetime.now() - start_sf
print("time taken by sf", loadtime_sf)


start_pt = datetime.datetime.now()
torch.load(pt_model)
loadtime_pt = datetime.datetime.now() - start_pt
print("time taken by pt", loadtime_pt)

print("final time taken", loadtime_pt/loadtime_sf)

time taken by sf 0:00:00.229092
time taken by pt 0:00:04.190145
final time taken 18.290228379864857


### Trying classification example

In [8]:
classifier = pipeline("sentiment-analysis")
classifier("here we go again for weath of the society")
classifier([

    "here is the cook, who cooks very good food",
    "he just fucks everything up"
])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


[{'label': 'POSITIVE', 'score': 0.9998283386230469},
 {'label': 'NEGATIVE', 'score': 0.9987642765045166}]

In [7]:
genrator = pipeline(task="text-generation")
genrator("life is hard")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'life is hard to do because there is always a chance of some sort of injury or injury that could take place. I\'m hoping we\'ve kept our head in the game and our players do our jobs," said coach Ron Francis. "We\'ll make'}]

####Tokenization

In [20]:
from transformers import AutoTokenizer

model = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer("this is a magic mashroom")

pt_batch = tokenizer(
    ["this is magic mashroom", "you will go to paradise after eating it"],
    padding=True,
    truncation=True,
    max_length=1024,
    return_tensors="pt"
    )
print(pt)

{'input_ids': tensor([[  101, 10372, 10127, 17800, 10321, 16158, 39639,   102,     0,     0],
        [  101, 10855, 11229, 11335, 10114, 27012, 10515, 59305, 10197,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


### taking model

In [21]:
from transformers import AutoModelForSequenceClassification

model = "nlptown/bert-base-multilingual-uncased-sentiment"
pt_model = AutoModelForSequenceClassification.from_pretrained(model)

In [24]:
pt_output = pt_model(**pt_batch)
print(pt_output)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1220, -1.6687, -0.5555,  0.6776,  2.1259],
        [-0.4588, -0.6722, -0.1753,  0.2649,  0.6359]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


###Train using pytorch

In [26]:
from torch import nn

pt_pred = nn.functional.softmax(pt_output.logits, dim=-1)
print(pt_pred)

tensor([[0.0285, 0.0165, 0.0502, 0.1722, 0.7327],
        [0.1222, 0.0987, 0.1622, 0.2519, 0.3651]], grad_fn=<SoftmaxBackward0>)


###saving model

In [28]:
pt_save_to_dir = "./data"
tokenizer.save_pretrained(pt_save_to_dir)
pt_model.save_pretrained(pt_save_to_dir)

#### using the existing model

In [33]:
pt_model_new = AutoModelForSequenceClassification.from_pretrained("./data")
pt_tokenizer_new = AutoTokenizer.from_pretrained("./data", form_tf=True)
pt_output_model = pt_model(**pt_batch)
print(pt_output_model)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.1220, -1.6687, -0.5555,  0.6776,  2.1259],
        [-0.4588, -0.6722, -0.1753,  0.2649,  0.6359]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
